In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from model import ode_y, ode_y_jump_approach_one
from utilities import find_nearest_value
from numba import njit

**ODE for y on (0, 2)**:
\begin{align*}
0 = \max_{\tilde{e}} \min_h &  -\delta \phi(y) + \delta \eta \log \tilde{e} \\
&+ \frac{d\phi(y)}{dy}\tilde{e}(\sum_{i=1}^n \pi_i^c \theta_i\ + \sigma_y \cdot h) + \frac{1}{2} \frac{d^2 \phi(y)}{(d y)^2}|\sigma_y|^2(\tilde{e})^2\\
&+ (\eta-1)(\gamma_1 + \gamma_2 y)\tilde{e}(\sum_{i=1}^n \pi_i^c \theta_i\ + \sigma_y \cdot h)\\
& + (\eta-1)\left[\frac{1}{2}\left(\gamma_2 + \gamma_2^+\boldsymbol{1}_{y\geq \bar{y}}\right)\right]|\sigma_y|^2(\tilde{e})^2\\
& +\frac{\xi_{1,m}}{2}h^\prime h + \xi_a\sum_{i=1}^{n} \pi_i^c(\log \pi_i^c - \log \hat{\pi}_i^c)\\
& + \mathcal{I}(y) \sum_{j=1}^m \hat{\pi}_j^d g_j [\phi_j(y)-\phi(y)] + \xi_{2,m} \mathcal{I}(y)\sum_{j=1}^m \hat{\pi}_j^d \left[1- g_j + g_j \log g_j\right]
\end{align*}

where
$$
\mathcal{I}(y) = \frac{1}{\sqrt{2\pi}\varsigma}\exp\{-\frac{(\bar{y}-y)^2}{2\varsigma^2}\}
$$

The FOC of $h$ gives:
$$
h^* = -\frac{\left(\frac{d\phi(y)}{dy} + (\eta-1)(\gamma_1 + \gamma_2 y )\right)\tilde{e}\sigma_y}{\xi_{1,m}}
$$


Substitute $h$ with $h^*$ in the ODE:

\begin{align*}
0 = \max_{\tilde{e}} & -\delta \phi(y) + \delta \eta \log \tilde{e} \\
& + G(y)\tilde{e}\sum_{i=1}^n \pi^c_i\theta_i + \frac{1}{2} \left(F(y)-\frac{G^2(y)}{\xi_{1,m}}\right)|\sigma_y|^2(\tilde{e})^2\\
&+ \xi_a\sum_{i=1}^{n} \pi_i^c(\log \pi_i^c - \log \hat{\pi}_i^c)\\
& + \mathcal{I}(y) \sum_{j=1}^m \hat{\pi}_j^d g_j [\phi_j(y)-\phi(y)] + \xi_{2,m} \mathcal{I}(y)\sum_{j=1}^m \hat{\pi}_j^d \left[1- g_j + g_j \log g_j\right]
\end{align*}

where $G(y):=\frac{d\phi(y)}{dy} + (\eta-1)(\gamma_1 + \gamma_2 y)$, $F(y):=\frac{d^2 \phi(y)}{(d y)^2} + (\eta-1)\left(\gamma_2 + \gamma_2^+\boldsymbol{1}_{y\geq \bar{y}}\right)$.

The FOC of $\tilde{e}$ gives:
$$
\tilde{e}^* = \frac{-\sum_{i=1}^n\pi^c_i \theta_i G(y) + \sqrt{\left(\sum_{i=1}^n \pi_i^c \theta_i\right)^2G^2(y)-4\delta \eta |\sigma_y|^2\left(F(y)-\frac{G^2(y)}{\xi_{1,m}}\right)}}{2|\sigma_y|^2\left(F(y)-\frac{G^2(y)}{\xi_{1,m}}\right)}
$$

In addition, the FOC of $\pi^c_i$ gives:
$$
\pi_i^c \propto \exp \left[-\frac{1}{\xi_a}G(y)\tilde{e}\theta_i\right]\hat{\pi}^c_i
$$

The FOC of $g_j$ gives:
$$
g_j = \exp \left(-\frac{1}{\xi_{1,m}}[\phi_j(y)-\phi(y)]\right)
$$

In [ ]:
ξ_1m = 1./400
ξ_a = 1000

η = .032
δ = .01

θ = pd.read_csv('../data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ)/len(θ)
σ_y = 1.2*1.86/1000

y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
γ_2p = np.array([0, .0197*2])
πd_o = np.array([.5, .5])
# γ_2p = np.array([0, .0197*2, .3853*2])
# πd_o = np.array([.475, .475, .05])

y_step = .02
y_grid_long = np.arange(0., 4., y_step)
y_grid_short = np.arange(0., 2+y_step, y_step)
n_bar = find_nearest_value(y_grid_long, y_bar) + 1

In [ ]:
# Prepare ϕ
ϕ_list = []
for γ_2p_i in γ_2p:
    model_paras = (η, δ, θ, πc_o, σ_y, ξ_1m, ξ_a, γ_1, γ_2, γ_2p_i, y_bar) 
    model_res = ode_y(y_grid_long, model_paras, v0=None, ϵ=.5,
                       tol=1e-8, max_iter=10_000, print_all=False)
    ϕ_list.append(model_res['v0'])

ϕ_list_short = []
for ϕ_i in ϕ_list:
#     temp = np.interp(y_grid_short, y_grid_long, ϕ_i)
    temp = ϕ_i[:n_bar]
    ϕ_list_short.append(temp)
ϕ_i = np.array(ϕ_list_short)

In [ ]:
# Compute ϕ
ς = .21
ξ_2m = ξ_1m*20
model_paras = (η, δ, θ, πc_o, σ_y, ξ_1m, ξ_2m, ξ_a, ς, γ_1, γ_2, y_bar, ϕ_i, πd_o)
model_res = ode_y_jump_approach_one(y_grid_short, model_paras, v0=ϕ_i[1],
                                                    ϵ=.5, tol=1e-9, max_iter=10000, print_all=True)

In [ ]:
plt.plot(y_grid_short, model_res['v0'])

In [ ]:
plt.plot(y_grid_short, model_res['e_tilde'])
plt.title(r'$\tilde{f}$')
plt.xlabel('y')

In [ ]:
plt.plot(y_grid_short, model_res['v0'])
plt.xlabel('y')
plt.title('Figure 3: value funciton, 174th iteration')
# plt.savefig('fig_3.pdf')

In [ ]:
# Compute worst case probabilities
πd_o_reshape = np.zeros((len(πd_o), len(y_grid_short)))
for i in range(πd_o_reshape.shape[0]):
    πd_o_reshape[i] = πd_o[i]
πd = [np.exp(1./ξ_2m*(ϕ-ϕ_i[i])) * πd_o[i] for i in range(len(πd_o))]
πd = np.array(πd)

In [ ]:
def simulation(y_grid, e_grid, π_grid, g_grid, θ=1.86/1000., y_start=.5394, T=50):
    Et = np.zeros(T+1)
    πt = np.zeros((len(π_grid), T+1))
    yt = np.zeros(T+1)
    gt = np.zeros_like(πt)
    for i in range(T+1):
        Et[i] = np.interp(y_start, y_grid, e_grid)
        for n in range(πt.shape[0]):
            πt[n, i] = np.interp(y_start, y_grid, π_grid[n])
            gt[n, i] = np.interp(y_start, y_grid, g_grid[n])
        yt[i] = y_start
        y_start = y_start + Et[i]*θ
    return Et, yt, πt, gt

In [ ]:
Et, yt, πt, gt = simulation(y_grid_short, e_tilde, πd, g, θ=1.86/1000., y_start=.5394, T=180)

In [ ]:
plt.plot(np.average(gt, axis=0, weights=πd_o))
plt.title('Approach one: proportional change in intensity')
plt.xlabel('Year')
plt.savefig('plot.pdf')

In [ ]:
plt.plot(πt[0], label='approach one')
# plt.plot(check[0], label='approach two')
plt.ylim(0, 0.5)
plt.title('Worst-case probability of low damage')
plt.legend()
plt.xlabel('Year')
plt.savefig('plot.pdf')

In [ ]:
plt.plot(yt)
plt.title('y')
plt.xlabel('Year')

In [ ]:
yt[-8]